# Cat vs Dog Classifier - Model Debugging

This notebook is designed to help diagnose issues with the cat vs dog classification model that's consistently predicting the same class regardless of the input image.

In [ ]:
# Import the necessary libraries
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os

# Configure TensorFlow
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

## 1. Load the Model

Let's first load the model and inspect its architecture

In [ ]:
# Load the model
model = tf.keras.models.load_model('model_v2.h5')

# Display model summary
model.summary()

## 2. Check the Model's Output Layer

Let's examine the output layer to confirm how many classes it's predicting and what activation function it uses

In [ ]:
# Get the last layer
output_layer = model.layers[-1]
print(f"Output layer: {output_layer}")
print(f"Output layer activation: {output_layer.activation.__name__ if hasattr(output_layer, 'activation') else 'No activation'}")
print(f"Output layer units: {output_layer.units if hasattr(output_layer, 'units') else 'Unknown'}")

## 3. Test with Sample Images

Let's test the model with known cat and dog images to see how it's behaving

In [ ]:
def preprocess_image(image_path):
    """Preprocess an image for model prediction"""
    # Open and convert to RGB
    img = Image.open(image_path).convert('RGB')
    # Resize to 128x128
    img = img.resize((128, 128))
    # Convert to numpy array
    img_array = np.array(img)
    # Normalize pixel values
    img_array = img_array.astype('float32') / 255.0
    # Reshape for model input
    img_array = np.reshape(img_array, (1, 128, 128, 3))
    return img_array, img

def predict_and_display(image_path):
    """Make a prediction on an image and display the results"""
    # Preprocess image
    processed_img, original_img = preprocess_image(image_path)
    
    # Make prediction
    prediction = model.predict(processed_img)
    
    # Get class probabilities
    class_0_prob = prediction[0][0]
    class_1_prob = prediction[0][1] if prediction.shape[1] > 1 else 1 - prediction[0][0]
    
    # Display results
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.imshow(original_img)
    plt.title(f"Image: {os.path.basename(image_path)}")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    labels = ['Class 0 (Cat)', 'Class 1 (Dog)']
    values = [class_0_prob, class_1_prob]
    
    plt.bar(labels, values)
    plt.title("Prediction Probabilities")
    plt.ylim(0, 1)
    
    plt.tight_layout()
    plt.show()
    
    # Print numerical results
    print(f"Raw prediction: {prediction}")
    print(f"Class 0 (Cat) probability: {class_0_prob:.4f}")
    print(f"Class 1 (Dog) probability: {class_1_prob:.4f}")
    print(f"Predicted class: {'Cat' if class_0_prob > class_1_prob else 'Dog'}")
    print("-" * 50)

# Test with sample images (replace these paths with actual images)
# test_paths = ["path_to_cat_image.jpg", "path_to_dog_image.jpg"]
# for path in test_paths:
#     predict_and_display(path)

## 4. Check Model's Class Indices

Let's verify the mapping between class indices and actual classes (cat/dog)

In [ ]:
# Try to load class indices if available
try:
    class_indices = model.class_indices
    print("Class indices:", class_indices)
except AttributeError:
    print("Model doesn't have class_indices attribute. This is common for models not created with ImageDataGenerator.")
    
    # Alternative: check if there's a separate file with class indices
    if os.path.exists('class_indices.npy'):
        class_indices = np.load('class_indices.npy', allow_pickle=True).item()
        print("Class indices loaded from file:", class_indices)
    else:
        print("No class indices information found.")
        print("Assuming binary classification with:")
        print("Index 0 = Cat")
        print("Index 1 = Dog")

## 5. Model Fix (if needed)

Based on debugging results, we might need to modify how we interpret the model's output

In [ ]:
# Test a fix by updating the interpretation of outputs
def fixed_prediction(image_path, swap_indices=False):
    """Make prediction with potentially fixed interpretation"""
    # Preprocess image
    processed_img, original_img = preprocess_image(image_path)
    
    # Make prediction
    prediction = model.predict(processed_img)
    
    # Get probabilities (potentially swapped)
    if swap_indices:
        cat_prob = prediction[0][1]  # Swapped - index 1 is cat
        dog_prob = prediction[0][0]  # Swapped - index 0 is dog
    else:
        cat_prob = prediction[0][0]  # Original - index 0 is cat
        dog_prob = prediction[0][1]  # Original - index 1 is dog
    
    # Print result
    result = "Cat" if cat_prob > dog_prob else "Dog"
    print(f"Image: {os.path.basename(image_path)}")
    print(f"Prediction: {result} (Cat: {cat_prob:.4f}, Dog: {dog_prob:.4f})")
    return result

# Example of testing with both interpretations
# image_path = "path_to_known_cat_image.jpg"
# print("Original interpretation:")
# fixed_prediction(image_path, swap_indices=False)
# print("\nSwapped interpretation:")
# fixed_prediction(image_path, swap_indices=True)

## 6. Updated App.py Code

Based on the debugging results, here's an updated version of the prediction code for app.py

In [ ]:
"""
# Code to update in app.py after determining correct class indices:

try:
    prediction = model.predict(processed_image)
    
    # Based on debugging results, use correct interpretation
    # Option 1: If original interpretation was correct
    cat_probability = float(prediction[0][0])
    dog_probability = float(prediction[0][1])
    
    # Option 2: If indices need to be swapped
    # cat_probability = float(prediction[0][1])
    # dog_probability = float(prediction[0][0])
    
    # Display results
    if dog_probability > cat_probability:
        result = f"Dog (Confidence: {dog_probability:.2%})"
        st.success(result)
    else:
        result = f"Cat (Confidence: {cat_probability:.2%})"
        st.success(result)
"""